# Вложенные функции и замыкания

Допустим, мы хотим решить такую **задачу**: есть функция, которая выполняет какие-то расчеты, и мы хотим вывести время ее выполнения.

In [1]:
import time

def calculate(x):
    res = 0;
    for i in range(x):
        res += (2 ** i) / x
    return res

t = time.time()
res = calculate(10)
print("res = {}, time = {:.5f}".format(res, time.time() - t))

res = 102.30000000000001, time = 0.00007


А если функция вызывается несколько раз? Наши друзья из солнечной Индии сделали бы так:

In [3]:
import time

def calculate(x):
    
    def get_result():
        res = 0;
        for i in range(x):
            res += (2 ** i) / x
        return res

    t = time.time()
    res = get_result()
    print("res = {}, time = {:.5f}".format(res, time.time() - t))

    return res

###
calculate(10)
calculate(20)
calculate(30)

res = 102.30000000000001, time = 0.00001
res = 52428.75, time = 0.00003
res = 35791394.099999994, time = 0.00001


35791394.099999994

Обратите внимание на 2 момента:
1. Мы объявили функцию `get_result` внутри `calculate`: по сути, `get_result` будет создаваться на лету каждый раз при вызове `calculate`.
2. Функция `get_result` обращается к переменной `x`, объявленной за пределами функции `get_result`.

Когда мы оказываемся в такое ситуации (т.е. на лету создаем, работающую с переменными извне), это называется **замыканием**.

**Замыкание** — это функция, которая динамически генерируется другой функцией, и они обе могут изменяться и запоминать значения переменных, которые были созданы (где-то) вне функции.


# Декораторы

А если таких функций `calculate` несколько? Можно вынести расчет времени в отдельную функцию. Обратите внимание, в функцию `logtime` мы передаем `calculate` и ее параметры для того, чтоб `calculate` была вызвана внутри функции `logtime`, иначе мы не сможет посчитать время ее выполнения.

In [4]:
import time

def logtime(f, *args, **kw):
    start = time.time()
    res = f(*args, **kw)
    print("res = {}, time = {:.5f}".format(res, time.time() - t))
    return res

def calculate(x):
    res = 0;
    for i in range(x):
        res += (2 ** i) / x
    return res

logtime(calculate, 10)
logtime(calculate, 20)
logtime(calculate, 30)

res = 102.30000000000001, time = 176.83764
res = 52428.75, time = 176.83776
res = 35791394.099999994, time = 176.83785


35791394.099999994

А если мы хотим, чтоб время выполнения выводилось всегда, а писать `logtime(calculate ... )` каждый раз нам лень? Мы можем описать функцию `logtime` так, чтоб она создавала (и возвращала) новую функцию-обертку, вызывающую наш `calculate`, и выполняющую нужные нам действия (подсчет времени).

А потом "подменить" наш `calculate` этой сгенерированной функцией-оберткой (т.е. результатом вызова `logtime`).


In [5]:
import time

def logtime(f):
    
    # объявляем функцию-обертку, которая вызывает функцию,
    # переданную через параметр f
    def wrapper(*args, **kw):
        start = time.time()
        res = f(*args, **kw)
        print("res = {}, time = {:.5f}".format(res, time.time() - t))
        return res

    return wrapper

def calculate(x):
    res = 0;
    for i in range(x):
        res += (2 ** i) / x
    return res

# замена calculate на функцию-обертку
calculate = logtime(calculate)

# здесь мы на самом деле вызываем уже не calculate,
# а функцию wrapper, созданную внутри logtime
calculate(10)
calculate(20)
calculate(30)

res = 102.30000000000001, time = 257.81418
res = 52428.75, time = 257.81427
res = 35791394.099999994, time = 257.81433


35791394.099999994

После всех этих манипуляций, мы превратили функцию `logtime` в **декоратор**.

**Декоратор** — это функция, которая принимает одну функцию в качестве аргумента и возвращает другую функцию.


In [6]:
@logtime
def calculate_2(x):
    res = 0;
    for i in range(x):
        res += (2 ** i) / x
    return res

calculate_2(10)
calculate_2(20)
calculate_2(30)

res = 102.30000000000001, time = 326.95899
res = 52428.75, time = 326.95907
res = 35791394.099999994, time = 326.95912


35791394.099999994


Т.е. **декораторы** — это, по сути, просто функции-обёртки, которые дают нам возможность делать что-либо до и после того, что сделает декорируемая функция, не изменяя её.


In [19]:
def bold(f):
    def wrapper(*args, **kw):
        return "<b>" + f(*args, **kw) + "</b>"
    return wrapper

def italic(f):
    def wrapper(*args, **kw):
        return "<i>" + f(*args, **kw) + "</i>"
    return wrapper

@bold
@italic
def say_hello():
    return "Hello!"

print(say_hello())

<b><i>Hello!</i></b>


In [11]:
def say_hello():
    return "Hello!"

say_hello = italic(say_hello)
say_hello = bold(say_hello)

print(say_hello())

<b><i>Hello!</i></b>


# Про `functools.wraps`

У функций есть свойства `__name__` и `__doc__`. Когда с помощью декоратора мы подменяем функцию, желательно копировать эти свойства в функцию обертку:

In [22]:
def decorate(f):
    def wrapper(*args, **kw):
        return f(*args, **kw)
    return wrapper

# @decorate
def hello():
    """ Возвращает какую-то глупую фразу """
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

hello
 Возвращает какую-то глупую фразу 


In [23]:
def decorate(f):
    def wrapper(*args, **kw):
        return f(*args, **kw)
    wrapper.__name__ = f.__name__
    wrapper.__doc__ = f.__doc__
    return wrapper

@decorate
def hello():
    """ Возвращает какую-то глупую фразу """
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

hello
 Возвращает какую-то глупую фразу 



Но есть более простой и красивый способ – использовать декоратор `functools.wraps`:

In [25]:
import functools

def decorate(f):
    @functools.wraps(f)
    def wrapper(*args, **kw):
        return f(*args, **kw)
    return wrapper

@decorate
def hello():
    """ Возвращает какую-то глупую фразу """
    return "Hello"

print(hello.__name__)
print(hello.__doc__)

hello
 Возвращает какую-то глупую фразу 


# Декораторы принимающие параметры

In [28]:
import functools

    def tag(t):
        def decorator_factory(f):
            @functools.wraps(f)
            def wrapper(*args, **kw):
                return t + f(*args, **kw) + t.replace("<", "</")
            return wrapper

        return decorator_factory

    @tag("<div>")
    @tag("<p>")
    def hello():
        """ Возвращает какую-то глупую фразу """
        return "Hello"

    print(hello())

<div><p>Hello</p></div>



Можно сделать параметры декоратора необязательными. Но в этом случае писать декоратор нужно будет все-равно со скобками:

In [29]:
import functools

def tag(t="<p>"):
    def decorator_factory(f):
        @functools.wraps(f)
        def wrapper(*args, **kw):
            return t + f(*args, **kw) + t.replace("<", "</")
        return wrapper

    return decorator_factory

@tag()
def hello():
    """ Возвращает какую-то глупую фразу """
    return "Hello"

print(hello())

<p>Hello</p>


# Декораторы классов

In [30]:
import functools

def singleton(cls):
    """ Декоратор, для преобразования класса в Singleton """
    instances = {}
    
    @functools.wraps(cls)
    def wrapper():
        if cls not in instances:
            instances[cls] = cls()
        return instances[cls]

    return wrapper

@singleton
class SomeService:
    def say_hello(self):
        print("Hello!")

obj1 = SomeService()
obj2 = SomeService()

print(obj1 is obj2)
print(id(obj1))
print(id(obj2))

True
4449754024
4449754024


# Что еще почитать?
* https://habrahabr.ru/post/141411/
* https://habrahabr.ru/post/141501/